In [17]:
# Training script
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd

In [18]:
# Load data
df = pd.read_csv("../Misc/generate_energy.csv")
df.drop(columns=["timestamp"], inplace=True)
df["hour_demand"] = df["hour"] * df["demand_prev_day"]


In [19]:
# Feature sets
demand_features = [
    "hour", "day_of_week", "month",
    "demand_prev_hour", "demand_prev_day", "demand_rolling_7d", "demand_rolling_24h",
    "hour_demand"
]
solar_features = [
    "hour", "day_of_week", "month", "holiday",
    "temperature", "cloud_cover"
]
diesel_features = [
    "hour", "day_of_week", "month", "holiday",
    "load_demand", "demand_prev_hour", "demand_prev_day", "demand_rolling_7d", "demand_rolling_24h",
    "solar_pv_output", "battery_soc"
]

In [20]:
#Split data for load_demand
X_demand = df[demand_features]
y_demand = df["load_demand"]
X_train_demand, X_test_demand, y_train_demand, y_test_demand = train_test_split(X_demand, y_demand, 
                                                                                test_size=0.2, random_state=42)

# Split data for solar_pv_output
X_solar = df[solar_features]
y_solar = df["solar_pv_output"]
X_train_solar, X_test_solar, y_train_solar, y_test_solar = train_test_split(X_solar, y_solar, 
                                                                            test_size=0.2, random_state=42)

# Split data for diesel_generator_usage
X_diesel = df[diesel_features]
y_diesel = df["diesel_generator_usage"]
X_train_diesel, X_test_diesel, y_train_diesel, y_test_diesel = train_test_split(X_diesel, y_diesel, 
                                                                                test_size=0.2, random_state=42)

In [21]:
def evaluate_model(y_test, y_pred):
    """Compute evaluation metrics."""
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    return {"MSE": mse, "RMSE": rmse, "R2": r2}


In [22]:

# Train models for load_demand
rf_demand = RandomForestRegressor(**rf_params)
rf_demand.fit(X_train_demand, y_train_demand)
joblib.dump(rf_demand, "random_forest_model.pkl")

['random_forest_model.pkl']

In [23]:
xgb_demand = XGBRegressor(**xgb_params)
xgb_demand.fit(X_train_demand, y_train_demand)
xgb_demand.save_model("xgboost_model.json")

In [24]:
# Train models for solar_pv_output
rf_solar = RandomForestRegressor(**rf_params)
rf_solar.fit(X_train_solar, y_train_solar)
joblib.dump(rf_solar, "rf_solar_model.pkl")

['rf_solar_model.pkl']

In [25]:
# Train models for diesel_generator_usage
rf_diesel = RandomForestRegressor(**rf_params)
rf_diesel.fit(X_train_diesel, y_train_diesel)
joblib.dump(rf_diesel, "rf_diesel_model.pkl")

['rf_diesel_model.pkl']

In [26]:
xgb_diesel = XGBRegressor(**xgb_params)
xgb_diesel.fit(X_train_diesel, y_train_diesel)
xgb_diesel.save_model("xgb_diesel_model.json")